In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.Container {width:85% !important;}
div.CodeMirror {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.output_area pre {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.output_wrapper pre {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
div.input {font-family:나눔고딕; font-size:13pt; line-height : 150%;}
</style>
"""))

# 각 포지션 별 세부 데이터 웹 크롤링

## 후스코어드에서 수비지표 가져오기

In [1]:
# ▶ 1. 필요한 패키지 로드
from selenium import webdriver # 동적 웹 크롤링
import time # 대기시간
import pandas as pd # 데이터 프레임화 해야하기 때문
import numpy as np
from pandas.io.html import read_html

In [131]:
# ▶ 2. 해당 리그 모든 선수의 수비 지표 크롤링
def defensive(url, sleep_time):  # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'D:/Gray_Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)

    time.sleep(sleep_time)
    defense = driver.find_element_by_link_text('Defensive')
    defense.click()

    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()

    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))

    df_defensive = pd.DataFrame(columns=['Player', 'Apps', 'Mins', 'Tackles', 'Inter', 'Fouls',
                                         'Offsides', 'Clear', 'Drb', 'Blocks', 'OwnG', 'Rating'])

    for i in np.arange(total_page):
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath(
            '//*[@id="statistics-table-defensive"]')
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')

        df2 = read_html(table_html)[0]  # 0번 인덱스에 모든 값이 들어가있음
        df_defensive = pd.concat([df_defensive, df2], axis=0)

        driver.find_element_by_link_text('next').click()  # 크롤링 후 다음페이지로 이동

    return(df_defensive)

url = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/8185/Stages/18594/PlayerStatistics/France-Ligue-1-2020-2021'
df = defensive(url, 2)

In [132]:
df.head()

,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player.1
0,"1NeymarPSG, 29, AM(CLR),FW",15(3),1417,0.8,0.5,1.8,-,-,1.6,-,-,7.69,"Neymar PSG, 29, AM(CLR),FW"
1,"2Kylian MbappéPSG, 22, AM(LR),FW",27(4),2389,0.1,0.1,0.7,-,0.1,0.2,-,-,7.68,"Kylian Mbappé PSG, 22, AM(LR),FW"
2,"3Andy DelortMontpellier, 29, AM(R),FW",30,2494,0.4,0.1,1.8,-,1.1,0.3,-,-,7.60,"Andy Delort Montpellier, 29, AM(R),FW"
3,"4Memphis DepayLyon, 27, AM(CL),FW",33(4),2932,0.1,0.1,0.8,-,0.1,0.5,-,-,7.56,"Memphis Depay Lyon, 27, AM(CL),FW"
4,"5Baptiste SantamaríaAngers, 26, DMC",3,270,3.0,2.3,2.3,-,2.3,1.7,-,-,7.38,"Baptiste Santamaría Angers, 26, DMC"


In [133]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1', 'Apps', 'Mins'], axis=1, inplace=True)

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"1NeymarPSG, 29, AM(CLR),FW",0.8,0.5,1.8,-,-,1.6,-,-,7.69
1,"2Kylian MbappéPSG, 22, AM(LR),FW",0.1,0.1,0.7,-,0.1,0.2,-,-,7.68
2,"3Andy DelortMontpellier, 29, AM(R),FW",0.4,0.1,1.8,-,1.1,0.3,-,-,7.60
3,"4Memphis DepayLyon, 27, AM(CL),FW",0.1,0.1,0.8,-,0.1,0.5,-,-,7.56
4,"5Baptiste SantamaríaAngers, 26, DMC",3.0,2.3,2.3,-,2.3,1.7,-,-,7.38


In [134]:
df.isna().sum()

Player      0
Tackles     0
Inter       0
Fouls       0
Offsides    0
Clear       0
Drb         0
Blocks      0
OwnG        0
Rating      0
dtype: int64

In [135]:
# Player에 이름만 넣기
spl = df['Player'].str.split(',')

print(spl)

0                       [1NeymarPSG,  29,  AM(CLR), FW]
1                 [2Kylian MbappéPSG,  22,  AM(LR), FW]
2            [3Andy DelortMontpellier,  29,  AM(R), FW]
3                [4Memphis DepayLyon,  27,  AM(CL), FW]
4               [5Baptiste SantamaríaAngers,  26,  DMC]
                             ...                       
579                 [580Habib KeitaLyon,  19,  Forward]
580        [581Anastasios DonisReims,  24,  AM(CR), FW]
581         [582Aleksandar RadovanovicLens,  27,  D(C)]
582              [583Marcin BulkaPSG,  21,  Goalkeeper]
583    [584Stefan BajicSaint-Etienne,  19,  Goalkeeper]
Name: Player, Length: 584, dtype: object


In [136]:
pls_list = []
for i in range(len(spl)):
    result_a = spl[i][0]
    pls_list.append(result_a)

df['Player'] = pls_list

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,1NeymarPSG,0.8,0.5,1.8,-,-,1.6,-,-,7.69
1,2Kylian MbappéPSG,0.1,0.1,0.7,-,0.1,0.2,-,-,7.68
2,3Andy DelortMontpellier,0.4,0.1,1.8,-,1.1,0.3,-,-,7.60
3,4Memphis DepayLyon,0.1,0.1,0.8,-,0.1,0.5,-,-,7.56
4,5Baptiste SantamaríaAngers,3.0,2.3,2.3,-,2.3,1.7,-,-,7.38


In [138]:
df.loc[250:280]

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
250,251Gautier LarsonneurBrest,-,-,-,-,0.8,-,-,-,6.55
251,252Mehdi AbeidNantes,1.9,1.1,1.7,-,0.6,0.8,0.1,-,6.55
252,253Sinaly DiomandéLyon,0.4,1.3,0.6,0.2,1.9,0.3,0.6,-,6.55
253,254Myziane MaolidaNice,0.7,0.3,0.9,-,0.2,0.4,0.1,-,6.55
254,255Pape GueyeMarseille,1.5,1.5,1.5,0.1,0.9,0.9,0.2,-,6.55
255,256Youcef AtalNice,1.4,1.4,1,0.2,0.6,0.7,0.1,-,6.55
256,257Arbër ZeneliReims,0.7,0.1,0.5,-,0.1,0.5,-,-,6.55
257,258Timothée KolodziejczakSaint-Etienne,1.2,1.5,1.2,0.3,3.3,0.3,0.6,-,6.55
258,259Dylan BronnMetz,1.5,1.2,1,0.3,2.8,0.5,0.4,-,6.55
259,260XekaLille,1,0.6,0.8,-,0.5,0.8,0.1,-,6.55


In [139]:
engla_teams = ['Man City', 'Liverpool', 'Chelsea', 'Man Utd', 'Arsenal', 'Tottenham', 'Huddersfield', 'Watford',
               'Bournemouth', 'Crystal Palace', 'Leicester', 'Wolves', 'Everton', 'Cardiff', 'Southampton', 'West Ham',
               'Fulham', 'Newcastle', 'Brighton', 'Burnley', 'Leeds', 'Norwich', 'Aston Villa', 'Sheff Utd', 'West Brom']

italy_teams = ['Juventus', 'Atalanta', 'AC Milan', 'Udinese', 'Roma', 'Napoli', 'Lazio', 'SPAL', 'Parma Calcio',
               'Sampdoria', 'Torino', 'Fiorentina', 'Sassuolo', 'Inter', 'Genoa', 'Cagliari', 'Bologna', 'Chievo', 'Empoli',
               'Frosinone', 'Brescia', 'Lecce', 'Verona', 'Crotone', 'Benevento', 'Spezia']

franc_teams = ['Montpellier', 'Nantes', 'Nimes', 'Saint-Etienne', 'PSG', 'Lille', 'Reims', 'Lyon', 'Rennes',\
               'Strasbourg', 'Guingamp', 'Dijon', 'Nice', 'Monaco', 'Marseille', 'Toulouse', 'Amiens', 'Bordeaux',\
               'Angers', 'Caen', 'Brest', 'Metz', 'Lorient', 'Lens', '']

germa_teams = ['Mainz', 'Leverkusen', 'Hoffenheim', 'Fortuna Duesseldorf', 'Eintracht Frankfurt', 'Freiburg',\
               'Werder Bremen', 'Augsburg', 'Borussia Dortmund', 'Bayern', 'Borussia M.Gladbach', 'Hannover', 'RBL',\
               'Nuernberg', 'Wolfsburg', 'Schalke', 'Stuttgart', 'Hertha Berlin', 'FC Koln', 'Union Berlin', 'Mainz',\
               'Paderborn']

spain_teams = ['Barcelona', 'Real Sociedad', 'Celta Vigo', 'Valencia', 'Real Madrid', 'Atletico', 'Athletic Bilbao', 'Espanyol',\
               'Levante', 'Eibar', 'Getafe', 'Real Betis', 'Leganes', 'Deportivo Alaves', 'Real Valladolid', 'SD Huesca', 'Sevilla',\
               'Villarreal', 'Girona', 'Rayo Vallecano', 'Osasuna', 'Mallorca', 'Granada', 'Cadiz', 'Elche']

In [3]:
def eng_players(name):
    for team in engla_teams:
        name = name.replace(team, "")
    return name

In [4]:
def ger_players(name):
    for team in germa_teams:
        name = name.replace(team, "")
    return name

In [88]:
def fra_players(name):
    for team in franc_teams:
        name = name.replace(team, "")
    return name

In [5]:
def italy_players(name):
    for team in italy_teams:
        name = name.replace(team, "")
    return name

In [6]:
def spain_players(name):
    for team in spain_teams:
        name = name.replace(team, "")
    return name

In [140]:
pls_list = []
for player in df['Player']:
    result_b = fra_players(player)
    pls_list.append(result_b)

df['Player'] = pls_list
df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,1Neymar,0.8,0.5,1.8,-,-,1.6,-,-,7.69
1,2Kylian Mbappé,0.1,0.1,0.7,-,0.1,0.2,-,-,7.68
2,3Andy Delort,0.4,0.1,1.8,-,1.1,0.3,-,-,7.60
3,4Memphis Depay,0.1,0.1,0.8,-,0.1,0.5,-,-,7.56
4,5Baptiste Santamaría,3.0,2.3,2.3,-,2.3,1.7,-,-,7.38


In [141]:
#  선수 이름 앞 숫자 자르기, 뒤 팀명 자르기
df['Player'] = df['Player'].replace(r'[0-9]', "", regex=True)

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Neymar,0.8,0.5,1.8,-,-,1.6,-,-,7.69
1,Kylian Mbappé,0.1,0.1,0.7,-,0.1,0.2,-,-,7.68
2,Andy Delort,0.4,0.1,1.8,-,1.1,0.3,-,-,7.60
3,Memphis Depay,0.1,0.1,0.8,-,0.1,0.5,-,-,7.56
4,Baptiste Santamaría,3.0,2.3,2.3,-,2.3,1.7,-,-,7.38


In [142]:
# '-'로 처리된 부분 NaN으로 처리하기
df = df.replace("-", "0.0")

df.head()

,Player,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Neymar,0.8,0.5,1.8,0.0,0.0,1.6,0.0,0.0,7.69
1,Kylian Mbappé,0.1,0.1,0.7,0.0,0.1,0.2,0.0,0.0,7.68
2,Andy Delort,0.4,0.1,1.8,0.0,1.1,0.3,0.0,0.0,7.60
3,Memphis Depay,0.1,0.1,0.8,0.0,0.1,0.5,0.0,0.0,7.56
4,Baptiste Santamaría,3.0,2.3,2.3,0.0,2.3,1.7,0.0,0.0,7.38


In [143]:
# csv파일로 저장하기
df.to_csv('D:/Gray_Bigdata\src/00_Projects/data/fra_2021_defensive_stats.csv')

## 후스코어드에서 미드필드 지표 가져오기

In [206]:
def passing(url, sleep_time):  # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'D:/Gray_Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)

    time.sleep(sleep_time)
    passing = driver.find_element_by_link_text('Passing')
    passing.click()

    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()

    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))

    df_passing = pd.DataFrame(columns=['Player', 'Apps', 'Mins', 'Assists', 'KeyP', 'AvgP',
                                         'PS%', 'Crosses', 'LongB', 'ThrB', 'Rating'])

    for i in np.arange(total_page):
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath(
            '//*[@id="statistics-table-passing"]')
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')

        df2 = read_html(table_html)[0]  # 0번 인덱스에 모든 값이 들어가있음
        df_passing = pd.concat([df_passing, df2], axis=0)

        driver.find_element_by_link_text('next').click()  # 크롤링 후 다음페이지로 이동

    return(df_passing)


url = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Stages/17835/PlayerStatistics/Italy-Serie-A-2019-2020'
df = passing(url, 2)


In [207]:
df.head()

,Player,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating,Player.1
0,"1Cristiano RonaldoJuventus, 36, M(L),FW",33,2919,5,1.5,36.5,85.1,0.2,0.9,-,7.82,"Cristiano Ronaldo Juventus, 36, M(L),FW"
1,"2Josip IlicicAtalanta, 33, AM(CLR),FW",21(5),1673,5,2.1,34.6,81.9,0.7,1.8,0.3,7.65,"Josip Ilicic Atalanta, 33, AM(CLR),FW"
2,"3Ciro ImmobileLazio, 31, FW",36(1),3176,9,1.6,23.9,77.0,-,0.9,0.1,7.57,"Ciro Immobile Lazio, 31, FW"
3,"4Luis AlbertoLazio, 28, AM(CL),FW",36,3122,15,2.9,57.6,83.5,1.2,3.1,0.4,7.55,"Luis Alberto Lazio, 28, AM(CL),FW"
4,"5Papu GómezAtalanta, 33, M(CL),FW",34(2),2861,16,2.8,47.8,85.3,1.8,2.1,0.1,7.55,"Papu Gómez Atalanta, 33, M(CL),FW"


In [208]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1', 'Apps', 'Mins'], axis=1, inplace=True)

df.head()

,Player,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,"1Cristiano RonaldoJuventus, 36, M(L),FW",5,1.5,36.5,85.1,0.2,0.9,-,7.82
1,"2Josip IlicicAtalanta, 33, AM(CLR),FW",5,2.1,34.6,81.9,0.7,1.8,0.3,7.65
2,"3Ciro ImmobileLazio, 31, FW",9,1.6,23.9,77.0,-,0.9,0.1,7.57
3,"4Luis AlbertoLazio, 28, AM(CL),FW",15,2.9,57.6,83.5,1.2,3.1,0.4,7.55
4,"5Papu GómezAtalanta, 33, M(CL),FW",16,2.8,47.8,85.3,1.8,2.1,0.1,7.55


In [209]:
df.isna().sum()

Player     0
Assists    0
KeyP       0
AvgP       0
PS%        0
Crosses    0
LongB      0
ThrB       0
Rating     0
dtype: int64

In [210]:
# Player에 이름만 넣기
spl = df['Player'].str.split(',')

print(spl)

0         [1Cristiano RonaldoJuventus,  36,  M(L), FW]
1           [2Josip IlicicAtalanta,  33,  AM(CLR), FW]
2                      [3Ciro ImmobileLazio,  31,  FW]
3               [4Luis AlbertoLazio,  28,  AM(CL), FW]
4               [5Papu GómezAtalanta,  33,  M(CL), FW]
                            ...                       
598       [599Raoul BellanovaAtalanta,  21,  Defender]
599               [600Jawad El YamiqGenoa,  29,  D(C)]
600    [601Lennart CzyborraAtalanta,  22,  D(L), M(L)]
601               [602Boris RadunovicVerona,  25,  GK]
602                [603RômuloGenoa,  34,  D(R), M(CR)]
Name: Player, Length: 603, dtype: object


In [211]:
pls_list = []
for i in range(len(spl)):
    result_a = spl[i][0]
    pls_list.append(result_a)

df['Player'] = pls_list
df.head()

,Player,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,1Cristiano RonaldoJuventus,5,1.5,36.5,85.1,0.2,0.9,-,7.82
1,2Josip IlicicAtalanta,5,2.1,34.6,81.9,0.7,1.8,0.3,7.65
2,3Ciro ImmobileLazio,9,1.6,23.9,77.0,-,0.9,0.1,7.57
3,4Luis AlbertoLazio,15,2.9,57.6,83.5,1.2,3.1,0.4,7.55
4,5Papu GómezAtalanta,16,2.8,47.8,85.3,1.8,2.1,0.1,7.55


In [212]:
#  선수 이름 앞 숫자 자르기, 뒤 팀명 자르기
df['Player'] = df['Player'].replace(r'[0-9]', "", regex=True)

df.head()

,Player,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Cristiano RonaldoJuventus,5,1.5,36.5,85.1,0.2,0.9,-,7.82
1,Josip IlicicAtalanta,5,2.1,34.6,81.9,0.7,1.8,0.3,7.65
2,Ciro ImmobileLazio,9,1.6,23.9,77.0,-,0.9,0.1,7.57
3,Luis AlbertoLazio,15,2.9,57.6,83.5,1.2,3.1,0.4,7.55
4,Papu GómezAtalanta,16,2.8,47.8,85.3,1.8,2.1,0.1,7.55


In [213]:
ls_list = []
for player in df['Player']:
    result_b = italy_players(player)
    ls_list.append(result_b)

df['Player'] = ls_list
df.head()

,Player,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Cristiano Ronaldo,5,1.5,36.5,85.1,0.2,0.9,-,7.82
1,Josip Ilicic,5,2.1,34.6,81.9,0.7,1.8,0.3,7.65
2,Ciro Immobile,9,1.6,23.9,77.0,-,0.9,0.1,7.57
3,Luis Alberto,15,2.9,57.6,83.5,1.2,3.1,0.4,7.55
4,Papu Gómez,16,2.8,47.8,85.3,1.8,2.1,0.1,7.55


In [214]:
df = df.replace("-", "0.0")

df.head()

,Player,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Cristiano Ronaldo,5,1.5,36.5,85.1,0.2,0.9,0.0,7.82
1,Josip Ilicic,5,2.1,34.6,81.9,0.7,1.8,0.3,7.65
2,Ciro Immobile,9,1.6,23.9,77.0,0.0,0.9,0.1,7.57
3,Luis Alberto,15,2.9,57.6,83.5,1.2,3.1,0.4,7.55
4,Papu Gómez,16,2.8,47.8,85.3,1.8,2.1,0.1,7.55


In [215]:
# csv파일로 저장하기
df.to_csv('D:/Gray_Bigdata/src/00_Projects/data/ita_1920_passing_stats.csv')

In [154]:
#################################################################
#################################################################
#################################################################

## 후스코어드에서 공격지표 가져오기

In [216]:
def offensive(url, sleep_time):  # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'D:/Gray_Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)

    time.sleep(sleep_time)
    offensive = driver.find_element_by_link_text('Offensive')
    offensive.click()

    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()

    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))

    df_offensive = pd.DataFrame(columns=['Player', 'Apps', 'Mins', 'Goals', 'Assists', 'SpG',
                                         'KeyP', 'Drb', 'Fouled', 'Off', 'Disp', 'UnsTch', 'Rating'])

    for i in np.arange(total_page):
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath(
            '//*[@id="statistics-table-offensive"]')
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')

        df2 = read_html(table_html)[0]  # 0번 인덱스에 모든 값이 들어가있음
        df_offensive = pd.concat([df_offensive, df2], axis=0)

        driver.find_element_by_link_text('next').click()  # 크롤링 후 다음페이지로 이동

    return(df_offensive)

url = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Stages/17835/PlayerStatistics/Italy-Serie-A-2019-2020'
df = offensive(url, 2)

In [217]:
df.head()

,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating,Player.1
0,"1Cristiano RonaldoJuventus, 36, M(L),FW",33,2919,31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82,"Cristiano Ronaldo Juventus, 36, M(L),FW"
1,"2Josip IlicicAtalanta, 33, AM(CLR),FW",21(5),1673,15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65,"Josip Ilicic Atalanta, 33, AM(CLR),FW"
2,"3Ciro ImmobileLazio, 31, FW",36(1),3176,36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57,"Ciro Immobile Lazio, 31, FW"
3,"4Luis AlbertoLazio, 28, AM(CL),FW",36,3122,6,15,2.7,2.9,2,1.1,-,1.3,1.4,7.55,"Luis Alberto Lazio, 28, AM(CL),FW"
4,"5Papu GómezAtalanta, 33, M(CL),FW",34(2),2861,7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55,"Papu Gómez Atalanta, 33, M(CL),FW"


In [218]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1', 'Apps', 'Mins'], axis=1, inplace=True)

df.head()

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,"1Cristiano RonaldoJuventus, 36, M(L),FW",31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82
1,"2Josip IlicicAtalanta, 33, AM(CLR),FW",15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65
2,"3Ciro ImmobileLazio, 31, FW",36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57
3,"4Luis AlbertoLazio, 28, AM(CL),FW",6,15,2.7,2.9,2,1.1,-,1.3,1.4,7.55
4,"5Papu GómezAtalanta, 33, M(CL),FW",7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55


In [219]:
df.isna().sum()

Player     0
Goals      0
Assists    0
SpG        0
KeyP       0
Drb        0
Fouled     0
Off        0
Disp       0
UnsTch     0
Rating     0
dtype: int64

In [220]:
# Player에 이름만 넣기
spl = df['Player'].str.split(',')

print(spl)

0         [1Cristiano RonaldoJuventus,  36,  M(L), FW]
1           [2Josip IlicicAtalanta,  33,  AM(CLR), FW]
2                      [3Ciro ImmobileLazio,  31,  FW]
3               [4Luis AlbertoLazio,  28,  AM(CL), FW]
4               [5Papu GómezAtalanta,  33,  M(CL), FW]
                            ...                       
598       [599Raoul BellanovaAtalanta,  21,  Defender]
599               [600Jawad El YamiqGenoa,  29,  D(C)]
600    [601Lennart CzyborraAtalanta,  22,  D(L), M(L)]
601               [602Boris RadunovicVerona,  25,  GK]
602                [603RômuloGenoa,  34,  D(R), M(CR)]
Name: Player, Length: 603, dtype: object


In [221]:
pls_list = []
for i in range(len(spl)):
    result_a = spl[i][0]
    pls_list.append(result_a)

df['Player'] = pls_list
df.head()

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,1Cristiano RonaldoJuventus,31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82
1,2Josip IlicicAtalanta,15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65
2,3Ciro ImmobileLazio,36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57
3,4Luis AlbertoLazio,6,15,2.7,2.9,2,1.1,-,1.3,1.4,7.55
4,5Papu GómezAtalanta,7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55


In [222]:
#  선수 이름 앞 숫자 자르기, 뒤 팀명 자르기
df['Player'] = df['Player'].replace(r'[0-9]', "", regex=True)

df.head()

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,Cristiano RonaldoJuventus,31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82
1,Josip IlicicAtalanta,15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65
2,Ciro ImmobileLazio,36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57
3,Luis AlbertoLazio,6,15,2.7,2.9,2,1.1,-,1.3,1.4,7.55
4,Papu GómezAtalanta,7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55


In [223]:
ls_list = []
for player in df['Player']:
    result_b = italy_players(player)
    ls_list.append(result_b)

df['Player'] = ls_list
df.head()

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,Cristiano Ronaldo,31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82
1,Josip Ilicic,15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65
2,Ciro Immobile,36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57
3,Luis Alberto,6,15,2.7,2.9,2,1.1,-,1.3,1.4,7.55
4,Papu Gómez,7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55


In [224]:
df = df.replace("-", "0.0")

df.head()

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,Cristiano Ronaldo,31,5,6.3,1.5,1.6,1.8,0.9,1,2.2,7.82
1,Josip Ilicic,15,5,3.1,2.1,2.3,2.3,0.1,1.8,2,7.65
2,Ciro Immobile,36,9,3.8,1.6,0.6,1,0.6,0.8,2.6,7.57
3,Luis Alberto,6,15,2.7,2.9,2,1.1,0.0,1.3,1.4,7.55
4,Papu Gómez,7,16,2.6,2.8,1.8,2.1,0.1,1.1,1.6,7.55


In [225]:
# csv파일로 저장하기
df.to_csv('D:/Gray_Bigdata/src/00_Projects/data/ita_1920_offensive_stats.csv')